In [ ]:
# Set up variables for agents
import os

PROJECT_ID = "your-gcp-project-id"  # @param {type:"string"}
LOCATION = "us-west1"  # @param {type:"string"}
DEPLOY_AGENT = "inventory_agent"

os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID
os.environ["GOOGLE_CLOUD_LOCATION"] = LOCATION


In [ ]:
# Run locally
!adk web

In [ ]:
# Deploy to GCP

!adk deploy agent_engine --project=$PROJECT_ID --region=$LOCATION inventory_agent --agent_engine_config_file=$DEPLOY_AGENT/.agent_engine_config.json

In [ ]:
import vertexai
from vertexai import agent_engines

vertexai.init(project=PROJECT_ID, location=LOCATION)

agents_list = list(agent_engines.list())
if agents_list:
    remote_agent = agents_list[0]  # Get the first (most recent) agent
    client = agent_engines
    print(f"✅ Connected to deployed agent: {remote_agent.resource_name}")
else:
    print("❌ No agents found. Please deploy first.")

In [ ]:
async for item in remote_agent.async_stream_query(
    message="What is the weather in Tokyo?",
    user_id="user_42",
):
    print(item)